In [ ]:
# -- select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
# -- where src = 'scale'
# -- and data_source in ('scale_hpi', 'scale_michelin', 'scale_fas', 'scale_bose')
# -- and inc_day in ('20211124', '20211117', '20211110', '20211103', 
# -- '20211027', '20211020', '20211013','20211006')
# -- and usage_flag = '1'

In [112]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

print(os.listdir('data_down'))


def allsundays(year):
    return pd.Series(pd.date_range(start=str(year), end=str(year+12), 
                        freq='2W-WED').strftime('%Y%m%d'))
fridays = tuple([i for i in list(allsundays(2021)[allsundays(2021) < date.today().strftime('%Y%m%d')][-8:])])



['1223_fifo_2wk.csv', '2week_sum.csv', 'dws_dsc_wh_fifo_alert_wi.csv', 'fifo_out_2w.csv', 'inv_1220.csv', 'inv_7.csv', 'inv_7_2.csv', 'inv_wmos.csv']


In [253]:
%%time
df = pd.read_csv('./data_down/inv_wmos.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[
    pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0
    ]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df
from datetime import date, timedelta

# sql = """
#     select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
#     where src = 'scale'
#     and ou_code in (
#     'HPPXXWHWDS', 
#     'MICHETCTGS'
#     'COSTASHHTS',
#     'ZEBRASHALS'
#     )
#     and inc_day in """+ str(fridays) + """
#     and usage_flag = '1'
#     """
# print(sql)

Wall time: 3.4 s


In [254]:
# code = ''
# df0 = pd.DataFrame()
# scan_len = len(df['inc_day'].unique())  # 8
# somelen = 8 - scan_len
# print([scan_len,somelen])

# def load_data(ou_code):
#     """
#     load bose data;
#     所有类型的qty都要加起来哦
#     只选择有多个收货日期的货物
#     """
#     global code, df, bose_inv, df0
#     df0 = pd.DataFrame()
#     df = df9
#     df = df[df['ou_code'].astype(str) == ou_code]

#     def fifo_fefo(df, type):
#         if type == 'fifo':
#             df['recived_date'] = pd.to_datetime(df['recived_date'])
#             df['fifo_fefo'] = 'fifo'
#         elif type == 'fefo':
#             df['recived_date'] = pd.to_datetime(df['expiration_date'].str.slice(0,10))
#             df['fifo_fefo'] = 'fefo'
#         else: 
#             pass
#         return df 

#     if ou_code in {'HPPXXWHWDS', 'HPPXXSHMGS'}:
#         # hp wh
#         print("load_data", " 'HPPXXWHWDS', 'HPPXXSHMGS' ")
#         code = '(QH|27|QI)'
#         df = fifo_fefo(df, 'fifo')

#     elif ou_code in {'MICHETCTGS', 'MICHESHXCS'}:
#         print("load_data", " 'MICHETCTGS', 'MICHESHXCS' ")
#         df = df[df['expiration_date'] != '4712-12-31']
#         # mich tc, rt,m  FEFO
#         code = '(BLOCKED_TH|RETURN)'
#         print("mich_if_if")
#         df = fifo_fefo(df, 'fefo')

#     elif ou_code == 'COSTASHHTS':
#         # COSTASHHTS expiration_date 没有空值.
#         df1 = df[df['expiration_date'] == '4712-12-31'] # fifo @# scale datetime. need redefine when other wms sys/..
#         df2 = df[df['expiration_date'] != '4712-12-31'] # fefo
#         df1 = fifo_fefo(df1, 'fifo')
#         df2 = fifo_fefo(df2, 'fefo')
#         df = pd.concat([df1, df2], axis = 0)
#         code = '(blocked)'

#     elif ou_code == 'SIEMESUEPS':
#         pass

#     # print(code)
    
    
#     df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location',\
#         'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
#         'recived_date','usage_flag', 'fifo_fefo','inc_day', 'ou_code']]
#     df['qty'] = df['on_hand_qty']
    
    
#     # 没有重复的 目前看....aaa
#     print("===============================oucode :: %s================================="%df['ou_code'].unique())
#     print(df['on_hand_qty'].describe())

#     df = df.sort_values(['sku_code', 'recived_date']).groupby(
#         ['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id', 'fifo_fefo'],
#         # dropna = False
#         ).agg(
#     {
#         'qty':'sum',
#         'location': set
#         # ? 这里如果group by location德华会有问题, 列数,miasjdijaisjd
#     }
#     ).sort_values(['sku_code', 'recived_date']).reset_index()
#     # 只选择有多个收货日期的货物
#     filter0 = df.groupby(['sku_code'])['recived_date'].agg(
#     {
#         set
#     }
#         ).reset_index()

#     filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
#     bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
#         .sort_values(['recived_date','sku_code', 'inc_day'])
#     bose_inv['ou_code'] = ou_code
#     print(bose_inv.head())
#     return bose_inv, code

# def snapshot():
#     """
#     pivot table. inc_day 快照 作为 cols
#     添加标记.
#     """
#     global df0, bose_inv
#     df0 = pd.DataFrame()
#     for i in bose_inv['sku_code'].unique():
#         df_out = bose_inv[bose_inv['sku_code'] == i]\
#             .pivot_table(columns='inc_day', index = 'recived_date', values='qty').reset_index()
#         df_out['sku_code'] = i
#         df0 = pd.concat([df0, df_out], axis = 0)
#     try:
#         df0.columns = df0.columns.get_level_values(level=1)
#     except:
#         pass

#     if re.search('(\d+)', str(df0.columns[0])) == None:
#         df0.columns = ['received_date', df0.columns[1],  df0.columns[2], 'sku']
#         df0 = df0[[df0.columns[1], df0.columns[2], 'received_date', 'sku']]
#         print('hah, mutated')
#     else:
#         print('ha. no need mutations')
#         pass

#     # if test.columns[0]  == '':
#     #     test.columns = ['received_date', test.columns[1],  test.columns[2], 'sku']
#     #     test = test[[test.columns[1], test.columns[2], 'received_date', 'sku']]
#     # else:
#     #     pass
#     print("snap_df0_column before in snap", df0.columns, "len of df0 in snap", (df0.shape))
#     df0 = df0.reset_index(drop = True) # 4 
#     print(df0.head())

#     # df0 = pd.DataFrame(np.zeros([3, 4]))
#     """
#     添加缺失列
#     """
#     if len(df0.columns) == 10:
#         print(list(df0.columns[0:scan_len]))
#         df0.columns = list(df0.columns[0:scan_len]) + ['received_date','sku']
#         print("normal process in snap::%s"%str(df0.shape))
#     else:
#         # pass
#         print("auto fill enabled , ncol is: %s" %(10 - len(df0.columns)))
#         # somelen = 10 - len(df0.columns)
#         # df_zero = pd.DataFrame(np.zeros([df0.shape[0], somelen]))
#         # df0 = pd.concat([df_zero, df0], axis = 1)
#         # df0.columns = list(
#         #     np.repeat(0, (somelen))
#         #     ) + list(
#         #         df0.columns[0:(scan_len)]
#         #         ) + ['received_date','sku']
#         names = df0.columns[0:2]
#         df_zero = pd.DataFrame(df0.iloc[:,0])
#         df_zero2 = pd.DataFrame()
#         # df_zero. copy()
#         for i in range(0,somelen):
#             df_zero2 = pd.concat([df_zero2, df_zero], axis = 1)

#         print(df_zero2.head())

#         df0 = pd.concat([df_zero2, df0], axis = 1)
#         print(df0.head())
#         df0.columns = list(range(11,11 + somelen)) + list(names) + ['received_date','sku']
#     scan_len = 8
#     print(df0.head())
#     df0 = df0.sort_values(['sku', 'received_date'])
#     df0['mark'] = 0
#     df0['mark'] = df0['mark'].where(
#         df0.iloc[:,  (scan_len - 1)].isna() == False, 'new')
#         # df0.iloc[:, 0: (scan_len - 1)].isna().all(axis = 1) == False, 'new')
#         # 
#     df0['mark'] = df0['mark'].where(
#         ~df0.iloc[:,(scan_len - 1)].isna() , 'clear')
#     # fill na~
#     df0 = df0.fillna(0)
#     bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
#         .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
#     bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
#     df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

#     # may lock
#     df0['mark'] = df0['mark'].where(df0.iloc[:, 0:scan_len].fillna(0).nunique(axis = 1) > 1, 'may_lock')
#     print("===========================snap!done for : %s=============================" %df0['ou_code'].unique())
#     return df0
# # print("===============================mid_function_check=================================")
# def err_part():
#     """
#     findout who are the naught peach.
#     err 中干掉了 new 干掉了maylock 
#     """
#     global df0

#     print("============================err_part!start: %s============================"%str(df0.shape))
#     scan_len = 8
#     df_err = df0[df0['mark'] != 'new']
#     # 补充可能被锁的标记
#     # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
#     # 干掉了maylock
#     df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
#     # print("===============================scan_len_err_function--%s================================="%scan_len)
#     # print(df_err.iloc[:,0:scan_len])
#     df_err['change'] = df_err.iloc[:,0:scan_len].diff(axis = 1).sum(axis = 1)
#     shift = df_err.groupby(['sku', 'wms_warehouse_id']).shift(1) 
#     shift = shift[['mark','change']]
#     shift.columns = ['lag_mark', 'lag_change']
#     shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
#     df_err = pd.concat([df_err, shift], axis = 1)
#     print("===============================err_part!done ::%s================================="%str(df_err.shape))
#     return df_err

# def output(df_err):
#     global df0
#     print("===============================output!start::%s================================="%str(df0.shape))

#     dishes = list(df_err[(df_err['lag_mark'] != 'clear') \
#         & (df_err['change'] < 0)
#         & (df_err['change'] != df_err['lag_change'])]['sku'].unique())
#     print("===============================output!done::%s================================="%str((df0[df0['sku'].isin(dishes)]).shape))

#     return df0[df0['sku'].isin(dishes)]

# def check(sku):
#     global df0
#     a = df0[df0['sku'].isin(sku)].sort_values(['sku','received_date'])
#     print("===============================check!done::%s================================="%str(a.shape))

#     return a 

# def ou_level_lock_codes(lock_code_to_eliminate):
#     """
#     正则. lock_code 需要被排除的, 依赖view表格. 
#     """ 
#     print("===========================ou_level_lock_codes!start!code :: %s============================="%str(code))
#     select_none_lock  = pd.DataFrame(
#         view.groupby('sku')[
#             'mark'
#             ].apply(list).astype(str).str.match('.+may')
#         ).reset_index()

#     select_none_lock2 = pd.DataFrame(
#         view.groupby('sku')[
#             'lock_codes'
#             ].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)
#         ).reset_index()
#     # 去重    
#     bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
#     bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
#     bose_err_list = list(bose_err_list.intersection(bose_err_list2))
#     bose_err_list = list(set(bose_err_list))
#     print("===========================ou_level_lock_codes!done :: %s==============================="%str(bose_err_list))

#     return bose_err_list


# # def printt(df):
# #     print('{note:=>50}'.format(note="shape") + '{note:=>50}'.format(note=df.shape))
# #     print(df.info())

# # # df.columns

# out_df = pd.DataFrame()
# for ou_code0 in df9['ou_code'].unique():

#     bose_inv = load_data(ou_code0)[0]
#     code = load_data(ou_code0)[1]
    
#     print('{note:=>50}'.format(note=ou_code0) + '{note:=>50}'.format(note=''))
#     print("===============================this_code: %s================================="%ou_code0)
#     print("===============================this_code_lock_code: %s================================="%code)


#     print("============================boseInv before snap==============================")
#     print(bose_inv.info())

#     df0 = snapshot()
#     print(df0.info())
#     df_err = err_part()

#     view = output(df_err)
#     bose_err_list = ou_level_lock_codes(code)
#     bose_definite_wrong = check(bose_err_list)
#     print("===============================~definite_wrong~=================================")
#     print(bose_definite_wrong.info())
#     out_df = pd.concat([out_df, bose_definite_wrong], axis = 0)
#     print(out_df.shape)

# print("===============================~loop_done~=================================")
# print(out_df.shape)
# print("===============================~'out_df.columns'~=================================")
# scan_len = 8
# out_df['start_week'] = fridays[0]
# out_df['end_week'] = fridays[-1]
# out_df.columns = [
#     str(j) + '_' + str(i) for i,j in enumerate(np.repeat('week', scan_len))
#     ] + [
#     'received_date','sku','mark','lock_codes',
#     'wms_warehouse_id','fifo_fefo','location','ou_code', 'start_of_week', 'end_of_week'
# ]
# out_df['inc_day'] = df9['inc_day'].max()
# out_df['location'] = [','.join(i) for i in out_df['location'] ]
# out_df['received_date'] = out_df['received_date'].astype(str)


In [257]:
df['ou_code'].unique()

array(['SIEMESUEPS', 'BURTSM071S', 'LUCKITJXQS', 'AEON1SSW2S'],
      dtype=object)

In [210]:
df9.query('sku_code == 2794452')[['lock_codes', 'inc_day']]

,lock_codes,inc_day
1951,BL,20211227
1952,BL,20211227
1953,BL,20211227
1954,BL,20211227
17277,NaN,20211227
...,...,...
162936,NaN,20220103
164597,NaN,20220103
164599,NaN,20220103
166632,NaN,20220103


In [ ]:
# out_df[out_df['ou_code'] == 'COSTASHHTS']

In [245]:
df = df9
df = df[df['ou_code'].astype(str) == 'SIEMESUEPS']

def fifo_fefo(df, type):
    if type == 'fifo':
        df['recived_date'] = pd.to_datetime(df['recived_date'])
        df['fifo_fefo'] = 'fifo'
    elif type == 'fefo':
        df['recived_date'] = pd.to_datetime(df['expiration_date'].str.slice(0,10))
        df['fifo_fefo'] = 'fefo'
    else: 
        pass
    return df 
df = fifo_fefo(df, 'fifo')
df = df[['wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 
        'recived_date','usage_flag', 'fifo_fefo','inc_day', 'ou_code']]
df['qty'] = df['on_hand_qty']

print(df['on_hand_qty'].describe())

df = df.sort_values(['sku_code', 'recived_date']).groupby(
    ['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id', 'fifo_fefo'],
    # dropna = False
    ).agg(
{
    'qty':'sum',
    'location': set
    # ? 这里如果group by location德华会有问题, 列数,miasjdijaisjd
}
).sort_values(['sku_code', 'recived_date']).reset_index()
# 只选择有多个收货日期的货物
filter0 = df.groupby(['sku_code'])['recived_date'].agg(
{
    set
}
    ).reset_index()

filter0 = pd.DataFrame(filter0[(filter0['set'].apply(len)) > 1]['sku_code'].drop_duplicates())
bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
ou_code = 'SIEMESUEPS'
bose_inv['ou_code'] = ou_code
bose_inv.head(3)

count    120902.000000
mean         56.453268
std         366.158393
min           0.000000
25%           1.000000
50%           7.000000
75%          36.000000
max       20200.000000
Name: on_hand_qty, dtype: float64


,sku_code,recived_date,lock_codes,inc_day,wms_warehouse_id,fifo_fefo,qty,location,ou_code
147,2801957,2021-01-14,QH,20211227,C21,fifo,38.0,{CM1AW0135},SIEMESUEPS
148,2801957,2021-01-14,QH,20220103,C21,fifo,38.0,{CM1AW0135},SIEMESUEPS
149,2801957,2021-01-19,QH,20211227,C21,fifo,15.0,{CM1AX0322},SIEMESUEPS


In [243]:
df0 = pd.DataFrame()
for i in bose_inv['sku_code'].unique():
    df_out = bose_inv[bose_inv['sku_code'] == i]\
        .pivot_table(columns='inc_day', index = 'recived_date', values='qty').reset_index()
    df_out['sku_code'] = i
    df0 = pd.concat([df0, df_out], axis = 0)
try:
    df0.columns = df0.columns.get_level_values(level=1)
except:
    pass


In [240]:
bose_inv.groupby(['sku_code', 'inc_day'])['lock_codes'].set()

AttributeError: 'SeriesGroupBy' object has no attribute 'set'

In [251]:
df0.iloc[:, 0:7].isna().sum(axis = 1) == 7

0      False
1      False
2      False
3      False
4      False
       ...  
415    False
416    False
417    False
418    False
419    False
Length: 420, dtype: bool

In [184]:
view = output(df_err)
view['sku'].unique()
# bose_err_list = ou_level_lock_codes(code)
# bose_definite_wrong = check(bose_err_list)

===============================output!start::(420, 16)=================================
===============================output!done::(34, 16)=================================


array([2794452, 2799892, 2802331, 2824105], dtype=int64)

In [198]:

bose_err_list = ou_level_lock_codes(code)
bose_definite_wrong = check(bose_err_list)

===========================ou_level_lock_codes!start!code :: IN=============================
===========================ou_level_lock_codes!done :: []===============================
===============================check!done::(0, 16)=================================


In [223]:
df0

,11,12,13,14,15,16,20211227,20220103,received_date,sku,mark,lock_codes,wms_warehouse_id,fifo_fefo,location,ou_code
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2021-12-15,2789990,clear,BL,C21,fifo,{CM1AJ0509},SIEMESUEPS
1,20.0,20.0,20.0,20.0,20.0,20.0,20.0,0.0,2021-12-23,2789990,clear,BL,C21,fifo,{CM1AN0551},SIEMESUEPS
2,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2021-12-20,2792058,may_lock,BL,C21,fifo,{CM1AG0138},SIEMESUEPS
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,2021-12-28,2792058,new,BL,C21,fifo,"{BFH0001, DMHBB0103}",SIEMESUEPS
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2022-01-03,2792058,new,BL,C21,fifo,{nan},SIEMESUEPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2021-12-17,5528730,clear,BL,C21,fifo,{CM1AN0213},SIEMESUEPS
416,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2021-12-20,5528730,may_lock,BL,C21,fifo,{CM1AP0227},SIEMESUEPS
417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2021-12-28,5528730,new,BL,C21,fifo,{CM1AN0110},SIEMESUEPS
418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2021-12-30,5528730,new,BL,C21,fifo,{CM1AB0228},SIEMESUEPS


In [219]:
df0

,11,12,13,14,15,16,20211227,20220103,received_date,sku,mark,lock_codes,wms_warehouse_id,fifo_fefo,location,ou_code
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2021-12-15,2789990,clear,BL,C21,fifo,{CM1AJ0509},SIEMESUEPS
1,20.0,20.0,20.0,20.0,20.0,20.0,20.0,0.0,2021-12-23,2789990,clear,BL,C21,fifo,{CM1AN0551},SIEMESUEPS
2,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2021-12-20,2792058,may_lock,BL,C21,fifo,{CM1AG0138},SIEMESUEPS
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,2021-12-28,2792058,new,BL,C21,fifo,"{BFH0001, DMHBB0103}",SIEMESUEPS
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2022-01-03,2792058,new,BL,C21,fifo,{nan},SIEMESUEPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2021-12-17,5528730,clear,BL,C21,fifo,{CM1AN0213},SIEMESUEPS
416,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2021-12-20,5528730,may_lock,BL,C21,fifo,{CM1AP0227},SIEMESUEPS
417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2021-12-28,5528730,new,BL,C21,fifo,{CM1AN0110},SIEMESUEPS
418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2021-12-30,5528730,new,BL,C21,fifo,{CM1AB0228},SIEMESUEPS


In [216]:
lock_code_to_eliminate = 'QX'
select_none_lock  = pd.DataFrame(
    view.groupby('sku')[
        'mark'
        ].apply(list).astype(str).str.match('.+may')
    ).reset_index()

select_none_lock2 = pd.DataFrame(
    view.groupby('sku')[
        'lock_codes'
        ].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)
    ).reset_index()
# 去重    
bose_err_list = set(select_none_lock['sku'].unique())
bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
bose_err_list = list(bose_err_list.intersection(bose_err_list2))
bose_err_list = list(set(bose_err_list))

In [218]:
bose_err_list

[2799892]

In [180]:
def ou_level_lock_codes(lock_code_to_eliminate):
    """
    正则. lock_code 需要被排除的, 依赖view表格. 
    """ 
    print("===========================ou_level_lock_codes!start!code :: %s============================="%str(code))
    select_none_lock  = pd.DataFrame(
        view.groupby('sku')[
            'mark'
            ].apply(list).astype(str).str.match('.+may')
        ).reset_index()

    select_none_lock2 = pd.DataFrame(
        view.groupby('sku')[
            'lock_codes'
            ].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)
        ).reset_index()
    # 去重    
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    print("===========================ou_level_lock_codes!done :: %s==============================="%str(bose_err_list))

    return bose_err_list

,11,12,13,14,15,16,20211227,20220103,received_date,sku,mark,lock_codes,wms_warehouse_id,fifo_fefo,location,ou_code
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2021-09-16,2794452,new,BL,C21,fifo,{DM3AY0449},SIEMESUEPS
20,162.0,162.0,162.0,162.0,162.0,162.0,162.0,24.0,2021-12-10,2794452,0,BL,C21,fifo,"{CRHBD0707, CM1AK0143}",SIEMESUEPS
21,162.0,162.0,162.0,162.0,162.0,162.0,162.0,24.0,2021-12-10,2794452,0,QX,C21,fifo,{CM1AC0357},SIEMESUEPS
22,12.0,12.0,12.0,12.0,12.0,12.0,12.0,0.0,2021-12-17,2794452,clear,BL,C21,fifo,{CM1AN0514},SIEMESUEPS
23,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,2021-12-20,2794452,may_lock,BL,C21,fifo,"{CM1AN0525, CM1AC0655, CM1AD0653}",SIEMESUEPS
24,16.0,16.0,16.0,16.0,16.0,16.0,16.0,0.0,2021-12-22,2794452,clear,BL,C21,fifo,{CM1AJ0429},SIEMESUEPS
25,16.0,16.0,16.0,16.0,16.0,16.0,16.0,0.0,2021-12-22,2794452,clear,QX,C21,fifo,{CM1AF0636},SIEMESUEPS
26,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,2021-12-23,2794452,may_lock,BL,C21,fifo,{CM1AL0507},SIEMESUEPS
27,63.0,63.0,63.0,63.0,63.0,63.0,63.0,28.0,2021-12-27,2794452,0,BL,C21,fifo,"{nan, CM1AA0114}",SIEMESUEPS
28,63.0,63.0,63.0,63.0,63.0,63.0,63.0,28.0,2021-12-27,2794452,0,QX,C21,fifo,"{CM1AA0202, CM1AE0227}",SIEMESUEPS


In [ ]:
  
        df0 = snapshot()
        print(df0.info())
        df_err = err_part()

        view = output(df_err)
        bose_err_list = ou_level_lock_codes(code)
        bose_definite_wrong = check(bose_err_list)
        print("===============================~definite_wrong~=================================")
        print(bose_definite_wrong.info())
        out_df = pd.concat([out_df, bose_definite_wrong], axis = 0)
        print(out_df.shape)
    except: 
        print('None data for code ::: %s :::'%ou_code0)

In [166]:
def load_data(ou_code):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    global code, df, bose_inv, df0 
    df0 = pd.DataFrame()
    df = df9
    df = df[df['ou_code'].astype(str) == ou_code]

    def fifo_fefo(df, type):
        if type == 'fifo':
            df['recived_date'] = pd.to_datetime(df['recived_date'])
            df['fifo_fefo'] = 'fifo'
        elif type == 'fefo':
            df['recived_date'] = pd.to_datetime(df['expiration_date'].str.slice(0,10))
            df['fifo_fefo'] = 'fefo'
        else: 
            pass
        return df 

    if ou_code in {'HPPXXWHWDS', 'HPPXXSHMGS'}:
        # hp wh
        print("load_data", " 'HPPXXWHWDS', 'HPPXXSHMGS' ")
        code = '(QH|27|QI)'
        df = fifo_fefo(df, 'fifo')

    elif ou_code in {'MICHETCTGS', 'MICHESHXCS'}:
        print("load_data", " 'MICHETCTGS', 'MICHESHXCS' ")
        df = df[df['expiration_date'] != '4712-12-31']
        # mich tc, rt,m  FEFO
        code = '(BLOCKED_TH|RETURN)'
        print("mich_if_if")
        df = fifo_fefo(df, 'fefo')

    elif ou_code == 'COSTASHHTS':
        # COSTASHHTS expiration_date 没有空值.
        df1 = df[df['expiration_date'] == '4712-12-31'] # fifo @# scale datetime. need redefine when other wms sys/..
        df2 = df[df['expiration_date'] != '4712-12-31'] # fefo
        df1 = fifo_fefo(df1, 'fifo')
        df2 = fifo_fefo(df2, 'fefo')
        df = pd.concat([df1, df2], axis = 0)
        code = '(blocked)'
        # wmos
    else:
        
        # make a date
        make_day = str(int(date.today().strftime('%Y')) + 50)+'-09-09'
        df1 = df[df['expiration_date'] == '9999-09-09'] # fifo @# scale datetime. need redefine when other wms sys/..
        df2 = df[df['expiration_date'] <= make_day] # fefo
        df1 = fifo_fefo(df1, 'fifo')
        df2 = fifo_fefo(df2, 'fefo')
        df = pd.concat([df1, df2], axis = 0)
        # code = '(\w)' # 不知道为什么这种方法不对, 但没时间调试了.
        code = "(PP|PV|BL|BA|QH|QX|LC|DW|PO|PT|DT|LW|PZ|PS)"


    # print(code)
    
    
    df = df[['wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 
        'recived_date','usage_flag', 'fifo_fefo','inc_day', 'ou_code']]
    df['qty'] = df['on_hand_qty']
    
    
    # 没有重复的 目前看....aaa
    print("===============================oucode :: %s================================="%df['ou_code'].unique())
    print(df['on_hand_qty'].describe())

    df = df.sort_values(['sku_code', 'recived_date']).groupby(
        ['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id', 'fifo_fefo'],
        # dropna = False
        ).agg(
    {
        'qty':'sum',
        'location': set
        # ? 这里如果group by location德华会有问题, 列数,miasjdijaisjd
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[(filter0['set'].apply(len)) > 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    bose_inv['ou_code'] = ou_code
    print(bose_inv.head())
    return bose_inv, code

def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    df0 = pd.DataFrame()
    for i in bose_inv['sku_code'].unique():
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns='inc_day', index = 'recived_date', values='qty').reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass

    if re.search('(\d+)', str(df0.columns[0])) == None:
        df0.columns = ['received_date', df0.columns[1],  df0.columns[2], 'sku']
        df0 = df0[[df0.columns[1], df0.columns[2], 'received_date', 'sku']]
        print('hah, mutated')
    else:
        print('ha. no need mutations')
        pass

    # if test.columns[0]  == '':
    #     test.columns = ['received_date', test.columns[1],  test.columns[2], 'sku']
    #     test = test[[test.columns[1], test.columns[2], 'received_date', 'sku']]
    # else:
    #     pass
    print("snap_df0_column before in snap", df0.columns, "len of df0 in snap", (df0.shape))
    df0 = df0.reset_index(drop = True) # 4 
    print(df0.head())

    # df0 = pd.DataFrame(np.zeros([3, 4]))
    """
    添加缺失列
    """
    if len(df0.columns) == 10:
        print(list(df0.columns[0:8]))
        df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
        print("normal process in snap::%s"%str(df0.shape))
    else:
        # pass
        print("auto fill enabled , ncol is: %s" %(10 - len(df0.columns)))
        # somelen = 10 - len(df0.columns)
        # df_zero = pd.DataFrame(np.zeros([df0.shape[0], somelen]))
        # df0 = pd.concat([df_zero, df0], axis = 1)
        # df0.columns = list(
        #     np.repeat(0, (somelen))
        #     ) + list(
        #         df0.columns[0:(scan_len)]
        #         ) + ['received_date','sku']
        names = df0.columns[0:2]
        df_zero = pd.DataFrame(df0.iloc[:,0])
        df_zero2 = pd.DataFrame()
        # df_zero. copy()
        for i in range(0,somelen):
            df_zero2 = pd.concat([df_zero2, df_zero], axis = 1)

        print(df_zero2.head())

        df0 = pd.concat([df_zero2, df0], axis = 1)
        print(df0.head())
        df0.columns = list(range(11,11 + somelen)) + list(names) + ['received_date','sku']
    scan_len = 8
    print(df0.head())
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
  
    # except:
    df0['mark'] = np.where(df0.iloc[:, 6].isna() == True, 'new', df0['mark'])
    df0['mark'] = np.where(df0.iloc[:, 7].isna() == True, 'clear', df0['mark'])
        
    
    # fill na~
    df0 = df0.fillna(0)
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

    # may lock
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:scan_len].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    print("===========================snap!done for : %s=============================" %df0['ou_code'].unique())
    return df0

In [ ]:

# print("===============================mid_function_check=================================")
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    global df0

    print("============================err_part!start: %s============================"%str(df0.shape))
    scan_len = 8
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    # print("===============================scan_len_err_function--%s================================="%scan_len)
    # print(df_err.iloc[:,0:scan_len])
    df_err['change'] = df_err.iloc[:,0:scan_len].diff(axis = 1).sum(axis = 1)
    if str(df_err['ou_code'].unique()) in {'HPPXXWHWDS', 'HPPXXSHMGS'}:
        shift = df_err.groupby(['sku', 'wms_warehouse_id', 'lock_code']).shift(1) 
    else:
        shift = df_err.groupby(['sku', 'wms_warehouse_id']).shift(1) 
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    print("===============================err_part!done ::%s================================="%str(df_err.shape))
    return df_err

def output(df_err):
    global df0
    print("===============================output!start::%s================================="%str(df0.shape))

    dishes = list(df_err[(df_err['lag_mark'] != 'clear') \
        & (df_err['change'] < 0)
        & (df_err['change'] != df_err['lag_change'])]['sku'].unique())
    print("===============================output!done::%s================================="%str((df0[df0['sku'].isin(dishes)]).shape))

    return df0[df0['sku'].isin(dishes)]

def check(sku):
    global df0
    a = df0[df0['sku'].isin(sku)].sort_values(['sku','received_date'])
    print("===============================check!done::%s================================="%str(a.shape))

    return a 

def ou_level_lock_codes(lock_code_to_eliminate):
    """
    正则. lock_code 需要被排除的, 依赖view表格. 
    """ 
    print("===========================ou_level_lock_codes!start!code :: %s============================="%str(code))
    select_none_lock  = pd.DataFrame(
        view.groupby('sku')[
            'mark'
            ].apply(list).astype(str).str.match('.+may')
        ).reset_index()

    select_none_lock2 = pd.DataFrame(
        view.groupby('sku')[
            'lock_codes'
            ].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)
        ).reset_index()
    # 去重    
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    print("===========================ou_level_lock_codes!done :: %s==============================="%str(bose_err_list))

    return bose_err_list


In [ ]:
# df9['ou_code'].unique()
bose_inv

In [120]:
ou_code0 = 'SIEMESUEPS'
bose_inv = load_data(ou_code0)[0]
code = load_data(ou_code0)[1]
#??????shabi?????

In [ ]:
df0 = pd.DataFrame()
for i in bose_inv['sku_code'].unique():
    df_out = bose_inv[bose_inv['sku_code'] == i]\
        .pivot_table(columns='inc_day', index = 'recived_date', values='qty').reset_index()
    df_out['sku_code'] = i
    df0 = pd.concat([df0, df_out], axis = 0)
try:
    df0.columns = df0.columns.get_level_values(level=1)
except:
    pass

if re.search('(\d+)', str(df0.columns[0])) == None:
    df0.columns = ['received_date', df0.columns[1],  df0.columns[2], 'sku']
    df0 = df0[[df0.columns[1], df0.columns[2], 'received_date', 'sku']]
    print('hah, mutated')
else:
    print('ha. no need mutations')
    pass

# if test.columns[0]  == '':
#     test.columns = ['received_date', test.columns[1],  test.columns[2], 'sku']
#     test = test[[test.columns[1], test.columns[2], 'received_date', 'sku']]
# else:
#     pass
print("snap_df0_column before in snap", df0.columns, "len of df0 in snap", (df0.shape))
df0 = df0.reset_index(drop = True) # 4 
print(df0.head())



In [ ]:

break #!@

In [ ]:
# df0 = pd.DataFrame(np.zeros([3, 4]))
"""
添加缺失列
"""
if len(df0.columns) == 10:
    print(list(df0.columns[0:8]))
    df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
    print("normal process in snap::%s"%str(df0.shape))
else:
    # pass
    print("auto fill enabled , ncol is: %s" %(10 - len(df0.columns)))
    # somelen = 10 - len(df0.columns)
    # df_zero = pd.DataFrame(np.zeros([df0.shape[0], somelen]))
    # df0 = pd.concat([df_zero, df0], axis = 1)
    # df0.columns = list(
    #     np.repeat(0, (somelen))
    #     ) + list(
    #         df0.columns[0:(scan_len)]
    #         ) + ['received_date','sku']
    names = df0.columns[0:2]
    df_zero = pd.DataFrame(df0.iloc[:,0])
    df_zero2 = pd.DataFrame()
    # df_zero. copy()
    for i in range(0,somelen):
        df_zero2 = pd.concat([df_zero2, df_zero], axis = 1)

    print(df_zero2.head())

    df0 = pd.concat([df_zero2, df0], axis = 1)
    print(df0.head())
    df0.columns = list(range(11,11 + somelen)) + list(names) + ['received_date','sku']
scan_len = 8
print(df0.head())


df0 = df0.sort_values(['sku', 'received_date'])
df0['mark'] = 0
df0['mark'] = df0['mark'].where(
    df0.iloc[:,  (scan_len - 1)].isna() == False, 'new')
    # df0.iloc[:, 0: (scan_len - 1)].isna().all(axis = 1) == False, 'new')
    # 
df0['mark'] = df0['mark'].where(
    ~df0.iloc[:,(scan_len - 1)].isna() , 'clear')
# fill na~
df0 = df0.fillna(0)
bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
    .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

# may lock
df0['mark'] = df0['mark'].where(df0.iloc[:, 0:scan_len].fillna(0).nunique(axis = 1) > 1, 'may_lock')

In [ ]:

out_df = pd.DataFrame()
for ou_code0 in df9['ou_code'].unique():

    bose_inv = load_data(ou_code0)[0]
    code = load_data(ou_code0)[1]
    
    print('{note:=>50}'.format(note=ou_code0) + '{note:=>50}'.format(note=''))
    print("===============================this_code: %s================================="%ou_code0)
    print("===============================this_code_lock_code: %s================================="%code)


    print("============================boseInv before snap==============================")
    print(bose_inv.info())

    try: 
        df0 = snapshot()
        print(df0.info())
        df_err = err_part()

        view = output(df_err)
        bose_err_list = ou_level_lock_codes(code)
        bose_definite_wrong = check(bose_err_list)
        print("===============================~definite_wrong~=================================")
        print(bose_definite_wrong.info())
        out_df = pd.concat([out_df, bose_definite_wrong], axis = 0)
        print(out_df.shape)
    except: 
        print('None data for code ::: %s :::'%ou_code0)

In [ ]:
bose_inv.loc[:, 'lock_codes']

In [ ]:
pd.DataFrame(
    bose_inv.groupby('sku_code')[
        'lock_codes'
        ].apply(list).astype(str).str.match('.+'+'(\w)')
    ).reset_index()

In [ ]:
re.search('(\d+)', str(test.columns[0])) == None

In [ ]:
if test.columns[0]  == '':
            test.columns = ['received_date', test.columns[1],  test.columns[2], 'sku']
            test = test[[test.columns[1], test.columns[2], 'received_date', 'sku']]
else:
    print('ha. noneed mutations')
    pass

In [ ]:
test 

In [ ]:
def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    df0 = pd.DataFrame()
    for i in bose_inv['sku_code'].unique():
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass
    print("snap_df0_column before in snap", df0.columns, "len of df0 in snap", (df0.shape))
    df0 = df0.reset_index(drop = True) # 4 
    # df0 = pd.DataFrame(np.zeros([3, 4]))
    """
    添加缺失列
    """
    if len(df0.columns) == 10:
        print(list(df0.columns[0:scan_len]))
        df0.columns = list(df0.columns[0:scan_len]) + ['received_date','sku']
        print("normal process in snap::%s"%str(df0.shape))
    else:
        # pass
        print("auto fill enabled , ncol is: %s" %(10 - len(df0.columns)))
        # somelen = 10 - len(df0.columns)
        # df_zero = pd.DataFrame(np.zeros([df0.shape[0], somelen]))
        # df0 = pd.concat([df_zero, df0], axis = 1)
        # df0.columns = list(
        #     np.repeat(0, (somelen))
        #     ) + list(
        #         df0.columns[0:(scan_len)]
        #         ) + ['received_date','sku']
        names = df0.columns[0:2]
        df_zero = pd.DataFrame(df0.iloc[:,0])
        df_zero2 = pd.DataFrame()
        # df_zero. copy()
        for i in range(0,somelen):
            df_zero2 = pd.concat([df_zero2, df_zero], axis = 1)

        print(df_zero2.head())

        df0 = pd.concat([df_zero2, df0], axis = 1)
        print(df0.head())
        df0.columns = list(range(11,11 + somelen)) + list(names) + ['received_date','sku']
    scan_len = 8
    print(df0.head())
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    df0['mark'] = df0['mark'].where(
        df0.iloc[:,  (scan_len - 1)].isna() == False, 'new')
        # df0.iloc[:, 0: (scan_len - 1)].isna().all(axis = 1) == False, 'new')
        # 
    df0['mark'] = df0['mark'].where(
        ~df0.iloc[:,(scan_len - 1)].isna() , 'clear')
    # fill na~
    df0 = df0.fillna(0)
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

    # may lock
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:scan_len].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    print("===========================snap!done for : %s=============================" %df0['ou_code'].unique())
    return df0
# print("===============================mid_function_check=================================")
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    global df0

    print("============================err_part!start: %s============================"%str(df0.shape))
    scan_len = 8
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    # print("===============================scan_len_err_function--%s================================="%scan_len)
    # print(df_err.iloc[:,0:scan_len])
    df_err['change'] = df_err.iloc[:,0:scan_len].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby(['sku', 'wms_warehouse_id']).shift(1) 
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    print("===============================err_part!done ::%s================================="%str(df_err.shape))
    return df_err

def output(df_err):
    global df0
    print("===============================output!start::%s================================="%str(df0.shape))

    dishes = list(df_err[(df_err['lag_mark'] != 'clear') \
        & (df_err['change'] < 0)
        & (df_err['change'] != df_err['lag_change'])]['sku'].unique())
    print("===============================output!done::%s================================="%str((df0[df0['sku'].isin(dishes)]).shape))

    return df0[df0['sku'].isin(dishes)]

def check(sku):
    global df0
    a = df0[df0['sku'].isin(sku)].sort_values(['sku','received_date'])
    print("===============================check!done::%s================================="%str(a.shape))

    return a 

def ou_level_lock_codes(lock_code_to_eliminate):
    """
    正则. lock_code 需要被排除的, 依赖view表格. 
    """ 
    print("===========================ou_level_lock_codes!start!code :: %s============================="%str(code))
    select_none_lock  = pd.DataFrame(
        view.groupby('sku')[
            'mark'
            ].apply(list).astype(str).str.match('.+may')
        ).reset_index()

    select_none_lock2 = pd.DataFrame(
        view.groupby('sku')[
            'lock_codes'
            ].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)
        ).reset_index()
    # 去重    
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    print("===========================ou_level_lock_codes!done :: %s==============================="%str(bose_err_list))

    return bose_err_list


In [ ]:
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    df_err['change'] = df_err.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby(['sku', 'wms_warehouse_id']).shift(1) 
    # q = []
    # for i in shift['mark']:
    #     if pd.isna(i):
    #         q.append('clear')
    #     elif i =='clear':
    #         q.append('clear')
    #     else:
    #         q.append('check')
    # shift['mark'] = q
    # shift['mark2'] = shift.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    # df_wrong = df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]
    # df_good  = df_err[(df_err['lag_mark'] != 'clear') | (df_err['change'] != 0)]
    return df_err
df_err = err_part()

In [ ]:
# df_err.to_pickle('./data_up/df_err_bose.p')
# df0.to_pickle('./data_up/df0_bose.p')
# df_err_bose = pd.read_pickle('./data_up/df_err_bose.p')
# df0_bose = pd.read_pickle('./data_up/df0_bose.p')
# # df0_bose = df0_bose[df0_bose['wms_warehouse_id'] == 'BOSE_SH']
# # df_err_bose = df_err_bose[df_err_bose['wms_warehouse_id'] == 'BOSE_SH']

In [ ]:
# pd.DataFrame(
#         view.groupby('sku')['mark'].apply(list).astype(str).str.contains('may') # may lock?
#         ).reset_index()

bose_inv['lock_codes'].unique()

In [ ]:

# 1000全新 3000退货 4000破损产品-

bose_err_list = ou_level_lock_codes('(QH|QI|27)')
bose_definite_wrong = check(bose_err_list, df0)
bose_definite_wrong['lock_codes'].unique()
    # .to_csv('./data_up/bose_fifo.csv', index = None, encoding = 'utf_8_sig')

=================================
# HP
=================================


In [ ]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('HPI_WH');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

In [ ]:
# df0_hp['lock_codes'].unique()

In [ ]:
df_err.to_pickle('./data_up/df_err_hpwh.p')
df0.to_pickle('./data_up/df0_hpwh.p')
df_err_hp = pd.read_pickle('./data_up/df_err_hpwh.p')
df0_hp = pd.read_pickle('./data_up/df0_hpwh.p')


In [ ]:
view = output(df_err_hp, df0_hp)
err_list_hp = ou_level_lock_codes('(QH|27)')

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0_hp)

# Z7Y79A 问一下
# 
hp_definite_wrong.head(6)

In [ ]:
# hp_definite_wrong[hp_definite_wrong['wms_warehouse_id']=='HPI_WH']

In [ ]:
hp_definite_wrong.to_csv('./data_up/hpwh_fifo.csv', index = None, encoding = 'utf_8_sig')

In [ ]:
# hp_definite_wrong[hp_definite_wrong['sku'] == '1VV21AA']

In [ ]:
hp_definite_wrong.to_csv('./data_up/hpsh_fifo.csv', index = None, encoding = 'utf_8_sig')

=================================
# michelin
=================================


In [ ]:
df0 = []
bose_inv = []

In [ ]:
df9[df9['wms_warehouse_name'] == 'OE']

In [ ]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('RT');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

In [ ]:
df_err.to_pickle('./data_up/df_err_michrt.p')
df0.to_pickle('./data_up/df0_michrt.p')
df_err_mich = pd.read_pickle('./data_up/df_err_michrt.p')
df0_mich = pd.read_pickle('./data_up/df0_michrt.p')


In [ ]:
output(df0=df0_mich, df_err=df_err_mich).head(3)

In [ ]:
# df0_mich['lock_codes'].str.slice(0,7).unique()

In [ ]:
view = output(df0=df0_mich, df_err=df_err_mich)
err_list_mich = ou_level_lock_codes('(BLOCKED_TH|RETURN)')

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]

mich_definite_wrong = check(err_list_mich, df0 = df0_mich)

# MATMICHELIN00591

# 同一时间 received 货物为什么会增加
# mich dot date?

In [ ]:
df0_mich[df0_mich['sku'] == '085753_111']

In [ ]:
mich_definite_wrong[mich_definite_wrong['sku'] == '085753_111']

In [ ]:
mich_definite_wrong.to_csv('./data_up/michrt_fifo.csv', index = None, encoding = 'utf_8_sig')

In [ ]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('OE');bose_inv
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df0=df0, df_err=df_err)
err_list_mich = ou_level_lock_codes('(BLOCKED_TH|RETURN)')
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]

mich_definite_wrong = check(err_list_mich, df0 = df0)

mich_definite_wrong.head()

In [ ]:
mich_definite_wrong.to_csv('./data_up/michoe_fifo.csv', index = None, encoding = 'utf_8_sig')

==========================
# fas <br>
==========================

In [ ]:
bose_inv = load_data('FAS') 
df0 = pd.DataFrame()
snapshot()

In [ ]:
df_err = err_part()
view = output(df_err, df0)

In [ ]:
bose_inv.head(3)

In [ ]:
err_list_hp = ou_level_lock_codes('(Avi)') 
bose_inv[bose_inv['sku'] == err_list_hp[0]]

In [ ]:

def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/fas.csv', index = None, encoding = 'utf_8_sig')

==========================
# part2 <br>
==========================

**select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
where src = 'scale'
and  ou_code in(
'FERRECDXXS',
'APPLESHWHW',
'HUSQVSHMFS',
'COSTASHHTS',
'FUJIXSYXXS',
'ZEBRASHALS'
)
and inc_day in ('20211124', '20211117', '20211110', '20211103', 
'20211027', '20211020', '20211013','20211006')
and usage_flag = '1'**

In [ ]:
%%time
df = pd.read_csv('./data_down/inv_7_2.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df

## Ferrero

In [ ]:
bose_inv = load_data('FERRECDXXS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp = ou_level_lock_codes('Blocked') 
bose_inv[bose_inv['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/ferrero_FERRECDXXS.csv', index = None, encoding = 'utf_8_sig')

## Apple sh

In [ ]:
df.ou_code.unique()

In [ ]:
bose_inv = load_data('APPLESHWHW') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp

In [ ]:
err_list_hp = ou_level_lock_codes('ttt') 
# bose_inv[bose_inv['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
# hp_definite_wrong[hp_definite_wrong['sku'] == '339S00668']

In [ ]:
hp_definite_wrong.to_csv('./data_up/apple_sh_APPLESHWHW.csv', index = None, encoding = 'utf_8_sig')

## costa 

In [ ]:
bose_inv = load_data('COSTASHHTS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp = ou_level_lock_codes('Held') 
df0[df0['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/costa_COSTASHHTS.csv', index = None, encoding = 'utf_8_sig')

## ZEBRASHALS

In [ ]:
bose_inv = load_data('ZEBRASHALS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

In [ ]:
err_list_hp = ou_level_lock_codes('HOLD1') 
df0[df0['sku'] == err_list_hp[0]]

In [ ]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

hp_definite_wrong.head(6)

In [ ]:
hp_definite_wrong.to_csv('./data_up/zebra_ZEBRASHALS.csv', index = None, encoding = 'utf_8_sig')